<a href="https://colab.research.google.com/github/gtejaswi13/bayeslab_agents_assignment/blob/main/agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install -q crewai 'crewai[tools]' langchain_groq --progress-bar off

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.17.1 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 5.29.2 which is incompatible.
tensorflow-metadata 1.13.1 requires protobuf<5,>=3.20.3, but you have protobuf 5.29.2 which is incompatible.
transformers 4.47.1 requires tokenizers<0.22,>=0.21, but you have tokenizers 0.20.3 which is incompatible.


In [8]:
from crewai import Agent, Task, Crew
from langchain_groq import ChatGroq

In [9]:
import os
from google.colab import userdata
groq_api_key = userdata.get('groq_api')
os.environ["GROQ_API_KEY"] = groq_api_key

serper_api_key = userdata.get('serper_api')
os.environ["SERPER_API_KEY"] = serper_api_key

In [10]:
import os
from crewai import LLM
groq_llm = LLM(
    model="groq/llama3-8b-8192",
    temperature=0.3,
    max_tokens=4096,
    api_key=groq_api_key,
)

In [11]:
# Define Agents
planner = Agent(
    role="Campaign Planner",
    goal="Plan a personalized marketing campaign for {product_name}.",
    backstory="You're tasked with developing a campaign strategy for the product: {product_name}. "
              "This includes defining target audiences, campaign objectives, unique selling points, "
              "and recommended promotional channels.",
    allow_delegation=False,
    verbose=True,
    llm=groq_llm
)

In [12]:
writer = Agent(
    role="Content Creator",
    goal="Write compelling and targeted marketing content for {product_name}.",
    backstory="You create persuasive marketing content based on the Campaign Planner's plan. "
              "Your content should highlight the product's benefits and appeal to the target audience.",
    allow_delegation=False,
    verbose=True,
    llm=groq_llm
)

In [13]:
editor = Agent(
    role="Content Editor",
    goal="Refine the marketing content to align with brand tone and optimize impact.",
    backstory="You review and refine the content created by the Content Creator, ensuring it adheres to "
              "brand guidelines, is engaging, and optimized for the target channels.",
    allow_delegation=False,
    verbose=True,
    llm=groq_llm
)

In [14]:
plan = Task(
    description=(
        "1. Research current market trends, target audience interests, "
        "and pain points for the product: {product_name}.\n"
        "2. Define unique selling points and outline promotional strategies.\n"
        "3. Identify effective marketing channels.\n"
        "4. Create a detailed content outline with key messaging points and a call to action."
    ),
    expected_output="A comprehensive marketing campaign plan document "
                    "with audience analysis, campaign objectives, content outline, "
                    "and promotional channel recommendations.",
    agent=planner,
)

In [15]:
write = Task(
    description=(
        "1. Use the campaign plan to create persuasive marketing content for {product_name}.\n"
        "2. Include the product's unique selling points and benefits.\n"
        "3. Ensure the tone aligns with the target audience and campaign objectives.\n"
        "4. Structure the content with an engaging introduction, key sections, and a clear call to action.\n"
        "5. Proofread for grammatical accuracy and brand voice alignment."
    ),
    expected_output="Well-crafted marketing content in markdown format, ready for publication.",
    agent=writer,
)

In [16]:
edit = Task(
    description=(
        "1. Review the marketing content for clarity, engagement, and adherence to brand tone.\n"
        "2. Ensure content is optimized for identified channels (e.g., social media, email, blogs).\n"
        "3. Refine structure and language to maximize impact and readability.\n"
        "4. Check for grammatical errors and inconsistencies."
    ),
    expected_output="Finalized marketing content optimized for publication.",
    agent=editor,
)

In [17]:
# Crew Workflow
crew = Crew(
    tasks=[plan, write, edit],
    allow_parallel=False  # Sequential workflow: Plan → Write → Edit
)

In [18]:
result = crew.kickoff(inputs={"product_name": "Electric Vehicle"})

# Agent: Campaign Planner
## Task: 1. Research current market trends, target audience interests, and pain points for the product: Electric Vehicle.
2. Define unique selling points and outline promotional strategies.
3. Identify effective marketing channels.
4. Create a detailed content outline with key messaging points and a call to action.


# Agent: Campaign Planner
## Final Answer: 
**Campaign Plan for Electric Vehicle**

**I. Research and Analysis**

* Target Audience:
	+ Demographics: Environmentally conscious individuals, urban dwellers, and tech-savvy consumers
	+ Psychographics: People who value sustainability, convenience, and innovation
* Market Trends:
	+ Growing demand for eco-friendly transportation
	+ Increasing government incentives for electric vehicle adoption
	+ Advancements in technology and range anxiety concerns
* Pain Points:
	+ Limited charging infrastructure
	+ Higher upfront cost compared to traditional vehicles
	+ Range anxiety and charging time concerns

**II

In [19]:
from IPython.display import Markdown
Markdown(str(result))

**Electrify Your Ride: Experience the Future of Transportation**

Are you ready to join the electric revolution? At [Brand Name], we're committed to providing you with a driving experience that's not only sustainable but also convenient and innovative. Our electric vehicles offer a zero-emission driving experience, reduced operating costs, and advanced technology features that will take your driving experience to the next level.

**Benefits of Electric Vehicles**

### Zero-Emission Driving Experience

Say goodbye to pollution and hello to a cleaner environment. Our electric vehicles produce zero tailpipe emissions, making them an excellent choice for those who care about the planet.

### Reduced Operating Costs

Electric vehicles are more cost-effective than traditional gasoline-powered vehicles. With lower maintenance costs and reduced fuel expenses, you'll save money in the long run.

### Advanced Technology and Features

Our electric vehicles come equipped with advanced technology features that will make your driving experience more enjoyable and convenient. From regenerative braking to advanced infotainment systems, our vehicles are designed to provide you with a driving experience like no other.

**Our Electric Vehicle Models**

### [Model Name]

Our [Model Name] is a compact electric vehicle that's perfect for city driving. With a range of up to [range] miles, you'll be able to get where you need to go without worrying about running out of charge.

### [Model Name]

Our [Model Name] is a mid-size electric vehicle that's designed for families and commuters. With a range of up to [range] miles, you'll be able to take your loved ones on long road trips without worrying about charging.

**Testimonials and User-Generated Content**

### "I was skeptical about electric vehicles at first, but after driving the [Model Name], I was blown away by its performance and range. It's the perfect car for my daily commute." - [Customer Name]

### "I love my [Model Name]! It's so easy to charge and maintain, and the advanced technology features make it feel like a luxury vehicle." - [Customer Name]

**Call to Action**

Ready to experience the future of transportation? Schedule a test drive today and discover the benefits of electric vehicles for yourself. Visit our website or contact your local dealership to learn more about our electric vehicle models and features.

**Additional Resources**

### [Blog Post] "The Benefits of Electric Vehicles: Why You Should Make the Switch"

### [Infographic] "Electric Vehicle Charging 101: A Guide to Charging Your Vehicle"

### [Video] "Electric Vehicle Test Drive: Experience the Future of Transportation"

**Contact Us**

Have questions about our electric vehicles or want to schedule a test drive? Contact us today and let us help you discover the benefits of electric vehicles for yourself.

[Brand Name]
[Address]
[Phone Number]
[Email Address]
[Website]

This finalized marketing content is optimized for publication, adhering to brand tone and guidelines, and is ready for distribution across identified channels.

In [20]:
!pip install gradio

In [21]:
import gradio as gr

In [22]:
# Gradio Interface function
def gradio_interface(product_name: str) -> str:
    try:
        # Run the multi-agent workflow for the marketing campaign
        result = crew.kickoff(inputs={"product_name": product_name})
        return result
    except Exception as e:
        return f"Error: {str(e)}"

In [23]:
# Gradio interface setup
interface = gr.Interface(
    fn=gradio_interface,
    inputs="text",
    outputs="text",
    title="AI Marketing Campaign Generator",
    description="Enter a product name to generate a comprehensive personalized marketing campaign."
)

In [24]:
# Launch the interface
if __name__ == "__main__":
    interface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f3c802ac5d1ac66c43.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
